# 1. Import Libraries

In [1]:
import time
import multiprocessing
import pandas as pd
import numpy as np
import random
from numpy import inf

# 2. Parameters

In [3]:
init_pheromone = 1  # Initial Pheromone
max_iter = 4        # Number of Iterations
max_transaction = 5 # Number of Transactions per Ant
ant = 5           # Number of Ants
rho = 0.05          # Pheromone Evaporation Rate
alpha = 1           # Pheromone Exponential Parameter
beta = 1            # Desirabillity Exponential Parameter
pheromone_scale = 1 # Pheromone Scaling

# 3. Data Sets

In [4]:
dataOL = pd.read_excel(r'sclp.xlsx', sheet_name='OrderList')
dataFR = pd.read_excel(r'sclp.xlsx', sheet_name='FreightRates')
dataWhCost = pd.read_excel(r'sclp.xlsx', sheet_name='WhCosts')
dataWhCap = pd.read_excel(r'sclp.xlsx', sheet_name='WhCapacities')

# 4. Data Preprocessing

In [5]:
#1 ORDER LIST

# 1.1 There is only one date in the order list, threfore it wont play any significant role for the optimization.
date_check = dataOL.groupby(['Order Date']).size()
print(date_check) 

# 1.2 Order Date, TPT, Ship ahead day count, Ship Late Day count will be removed as they are not within the scope of this optimization.
dataOL=dataOL.drop(columns=['Order Date','TPT','Ship ahead day count','Ship Late Day count'])

# 1.3 carrier_check is to check the distinct value of carriers. Which then will be used to filter only the necessary carriers for the transportation.
carrier_check = dataOL.groupby(['Carrier']).size().reset_index()
carrier_list = pd.Series({cc: carrier_check[cc].unique() for cc in carrier_check})

# 1.4 port_check is to check the distinct value of ports. Which then will be used to filter only the necessary origin ports for the transportation.
port_check = dataOL.groupby(['Origin Port']).size().reset_index()
port_list = pd.Series({pc: port_check[pc].unique() for pc in port_check})

# 1.5 plant_check is to check the distinct value of warehouses. Which then will be used to filter only the necessary warehouses for the transportation.
#     It will also be used to filter the warehouse needed to use based on the Warehouse Cost Dataframe and also Warehouse Capacity DataFrame.
plant_check = dataOL.groupby(['Plant Code']).size().reset_index()
plant_list = pd.Series({pc: plant_check[pc].unique() for pc in plant_check})

# [2 FREIGHT RATE]
# 2.1 Filter the Freight Rate DataFrame to only get the necessary carriers to the existing Order List DataFrame.
dataFR = dataFR[dataFR['Carrier'].isin(carrier_list['Carrier'])]

# 2.2 Filter the Freight Rate DataFrame to only get the necessary carriers to the existing Order List DataFrame.
dataFR = dataFR[dataFR['orig_port_cd'].isin(port_list['Origin Port'])]

# 2.3 Drop out tpt_day_cnt and carrier type as they are not needed for the optimization.
dataFR = dataFR.drop(columns=['tpt_day_cnt','Carrier type'])

# 3 [WAREHOUSE COST]
# 3.1 Filter the Warehouse Cost DataFrame to only get the necessary warehouses to the existing Order List DataFrame.
dataWhCost = dataWhCost[dataWhCost['WH'].isin(plant_list['Plant Code'])]

# 4 [WAREHOUSE CAPACITY]
# 4.1 Filter the Warehouse Cost DataFrame to only get the necessary warehouses to the existing Order List DataFrame.
dataWhCap = dataWhCap[dataWhCap['Plant ID'].isin(plant_list['Plant Code'])]


Order Date
2013-05-26    9215
dtype: int64


# 5. Main Function

In [ ]:
#Pheromone Initialisation
dataOL['Pheromone'] = init_pheromone
dataFR['Pheromone'] = init_pheromone

#The number of Iterations of the Ant Colony.
for colony_iter in range(max_iter):

    #The number of Ants in a single Colony Iteration.
    processes= []
    res = []
    res_aco_t = multiprocessing.Queue()

    for _ in range(ant):

        p = multiprocessing.Process(target=aco_traversal, args=(colony_iter,max_transaction,init_pheromone,res_aco_t))
        p.start()
        processes.append(p)
        
    for process in processes:
        process.join()

    for _ in range(ant):
        x = res_aco_t.get()
        res.append(x)

    fitness = objective_function(res)
    dataOL,dataFR = pheromone_update(res,fitness)
    # print(dataOL.where(dataOL['Pheromone']>0).dropna())
    



# 6. Funtions

## 6.1 ACO Traversal Function

In [15]:
# def aco_traversal(max_transaction,var):
def aco_traversal(colony_iter,max_transaction,init_pheromone,res_aco_t):
    p = multiprocessing.current_process()
    print('[ANT ENTERING]')
    current_transaction = 0 # The current Order Transaction for an Ant.
    result = []
    total_cost = 0
    dataOL_sub, dataFR_sub, dataWhCap_sub, dataWhCost_sub = fresh_dataframes(dataOL, dataFR, dataWhCap, dataWhCost)

    dataOL_sub['Status'] = True
    dataFR_sub['Status'] = True
    potential_transaction = pd.DataFrame()
    order_carrier_record = pd.DataFrame()
    

    #The Loop Will Go On Until Max Number of Transactions Has Been Reached.
    while current_transaction < max_transaction:
        print(current_transaction)
        route_status = True
        use_carrier = True                      #Does the logistics requires carriers?
        order_carrier_status = False            #Has the carrier been used?
        carrier_index,carrier_index_index = 0,0 # Index for The Carrier and The Index's Index

        f_data_ol_status = (dataOL_sub['Status'] == True) # [FILTER]

        #PORT SELECTION SEGMENT
        p_origin_port_availability = dataOL_sub.where(f_data_ol_status).groupby(['Origin Port','Status'])['Pheromone'].sum().reset_index()
        selected_port = selection(p_origin_port_availability,'[PORT SELECTION]')    # [SELECTION] Selection Process for PORT  

        #PLANT CODE SELECTION SEGMENT
        p_plant_availability = dataOL_sub.where(f_data_ol_status).groupby(['Origin Port','Plant Code','Status','Pheromone']).size().reset_index(name='Availability')
        f_origin_port = (p_plant_availability["Origin Port"] == selected_port) # [FILTER] Selected Origin Port Filter
        p_plant_availability = p_plant_availability.where(f_origin_port).groupby(['Plant Code','Status'])['Pheromone'].sum().reset_index()
        selected_plant_code = selection(p_plant_availability,'[PLANT SELECTION]') # [SELECTION] Selection Process for PLANT CODE

        #ORDER ID SELECTION SEGMENT
        f_plant_from_dataOL = (dataOL_sub["Plant Code"] == selected_plant_code) # [FILTER] Selected Plant Filter DataOL
        f_port_from_dataOL = (dataOL_sub["Origin Port"] == selected_port) # [FILTER] Selected Origin Port Filter DataOL
        path_order_availability = dataOL_sub.where(f_plant_from_dataOL & f_port_from_dataOL & f_data_ol_status).groupby(['Order ID','Service Level','Carrier','Status'])['Pheromone'].sum().reset_index()
        selected_path_order = selection(path_order_availability,'[ORDER ID SELECTION]')
        # print([selected_path_order['Order ID'][0]])
        
        #CARRIER SELECTION SEGMENT
        f_carrier_carrier = (dataFR_sub['Carrier'] == selected_path_order['Carrier'][0]) # [FILTER]
        f_carrier_order_origin = (dataFR_sub['orig_port_cd'] == selected_port) # [FILTER]

        if selected_path_order['Carrier'][0] != 'V44_3':
            f_carrier_sel = (dataFR_sub['Status'] == True)
            carrier_availability = dataFR_sub.where(f_carrier_order_origin & f_carrier_sel).groupby([dataFR_sub.index,'Carrier','orig_port_cd','svc_cd','minm_wgh_qty','max_wgh_qty','Status'])['Pheromone'].sum().reset_index()
            selected_carrier = selection(carrier_availability,'[CARRIER SELECTION]')
        else:
            use_carrier = False #No Carriers Needed
            selected_carrier = 'CRF'
        
        # print(selected_port,selected_plant_code,selected_path_order)
        if use_carrier:
            # [SINGLE ROW DATAFRAME]
            potential_transaction = pd.DataFrame(   {'Port': [selected_port],
                                                    'Plant Code' : [selected_plant_code],
                                                    'Order ID' : [selected_path_order['Order ID'][0]],
                                                    'Service ID' : [selected_path_order['Service Level'][0]],
                                                    'Carrier' : [selected_path_order['Carrier'][0]],
                                                    'Carrier Index' : [selected_carrier['level_0'][0]],
                                                    'Status' : [True],
                                                    'Pheromone' : 0,
                                                    'Cost' : 0})

            if order_carrier_record.empty is not True:

                # [FILTERS]
                f_ocr_port = order_carrier_record['Port'] == selected_port
                f_ocr_plant = order_carrier_record['Plant Code'] == selected_plant_code
                f_ocr_order = order_carrier_record['Order ID'] == selected_path_order['Order ID'][0]
                f_ocr_service = order_carrier_record['Service ID'] == selected_path_order['Service Level'][0]
                f_ocr_carrier = order_carrier_record['Carrier'] == selected_path_order['Carrier'][0]
                f_ocr_carrier_index = order_carrier_record['Carrier Index'] == selected_carrier['level_0'][0]
                f_ocr_status = order_carrier_record['Status'] == True
                
                ocr = order_carrier_record.where(f_ocr_port & f_ocr_plant & f_ocr_order & f_ocr_service & f_ocr_carrier & f_ocr_carrier_index).dropna()
                if ocr.empty:
                    order_carrier_record = pd.concat([order_carrier_record,potential_transaction], ignore_index = True, axis = 0)
                else:
                    route_status = False
                    
            else:
                order_carrier_record = pd.concat([order_carrier_record,potential_transaction], ignore_index = True, axis = 0)

        else:
            # [SINGLE ROW DATAFRAME]
            potential_transaction = pd.DataFrame(   {'Port': [selected_port],
                                                    'Plant Code' : [selected_plant_code],
                                                    'Order ID' : [selected_path_order['Order ID'][0]],
                                                    'Service ID' : [selected_path_order['Service Level'][0]],
                                                    'Carrier' : 'No Carrier',
                                                    'Carrier Index' : 'No Carrier',
                                                    'Status' : [True],
                                                    'Pheromone' : 0,
                                                    'Cost' : 0})
            
            order_carrier_record = pd.concat([order_carrier_record,potential_transaction], ignore_index = True, axis = 0)

        # COST CALCULATION
        if use_carrier:
            #kira t cost
            #order_carrier_record
            t_cost = transportation_cost(selected_path_order,selected_carrier,dataOL_sub,dataFR_sub)
            
            if type(t_cost) is str:
                route_status = False
            else:
                total_cost += t_cost
            #kira whcost''
        
        wh_cost = warehouse_cost(dataWhCost_sub,dataWhCap_sub,selected_path_order,selected_plant_code,dataOL_sub)
        
        if type(wh_cost) is str:
            route_status = False
        else:
            total_cost += wh_cost

        #AVAILABILITY UPDATE
        if route_status:

            # UPDATE FOR ORDER ID
            f_data_OL_index = dataOL[dataOL['Order ID']==selected_path_order['Order ID'][0]].index.values #filter index finding
            dataOL_sub.at[f_data_OL_index,'Status'] = False

            #UPDATE FOR WAREHOUSE CAP
            f_wh_cap_index = dataWhCap_sub[dataWhCap_sub['Plant ID']==selected_plant_code].index.values #filter index finding
            counter = dataWhCap_sub['Daily Capacity '][f_wh_cap_index]-1
            dataWhCap.at[f_wh_cap_index,'Daily Capacity '] = counter
            current_transaction+=1
            # print(current_transaction)

            order_carrier_record.iloc[-1,order_carrier_record.columns.get_loc('Cost')] = total_cost

            if use_carrier:

                # UPDATE FOR FRIEGHT RATE
                dataFR_sub .at[selected_carrier['level_0'][0],'Status'] = False

        else:

            order_carrier_record = order_carrier_record.head(order_carrier_record.shape[0] -1)

    result = [order_carrier_record]
    res_aco_t.put(result)
    print('[ANT DONE]')

## 6.2 Selection Function

In [8]:
def selection(dat,type_sel):

    cumulative_pheromone,cum_p_segment = 0,0    # Total Pheromones and Each Segmentation of Pheromone.
    start_range,end_range = 0,0                 # Range of Each Segmented Pheromone.
    data_length = 0                             # Length of Data to be segmentized.
    selection_list = []                         # Selection of choices to be segmentized.
    roulette_value = random.uniform(0, 1)       # Roulette Value.
    target = 0                                  # The Target answer after the Roullete Wheel.

    # 1. Eliminate Routes With False Status.
    f_status_count = dat['Status'] == True          # [FILTER] Filter the status to only True.
    dat = dat.where(f_status_count).dropna()        # Filter the data status to only True and drop NaN values.
    data_length = len(dat.index)                    # Find the length of data.
    cumulative_pheromone = dat['Pheromone'].sum()   # Accumulate Pheromone for Selection.
    
    #2. Normalize the Routes in a Uniform Roulette Wheel.
    for index,row in dat.iterrows():
        cur_list=[]

        # 2.1 First ACO Iteration or specfically when the Accumulated Pheromone is 0.
        if cumulative_pheromone == 0:
            end_range = (index+1)/data_length

        # 2.2 next Iterations.
        else:
            end_range = (cum_p_segment+row['Pheromone']/cumulative_pheromone)
            cum_p_segment += row['Pheromone']

        # 2.3 Segmentize by range of index/pheromone.
        cur_list = [row[0],start_range,end_range]
        selection_list.append(cur_list)
        start_range = end_range

    # 3. Applying the Roulette Wheel to get the Target answer. 
    for sel_ in selection_list:
        if sel_[1] <= roulette_value and sel_[2] >= roulette_value:
            probability = "{:.2f}".format((sel_[2]-sel_[1])*100)
            target = sel_[0]

    # 4. If the Data is based on Order ID.
    if dat.columns[0] == 'Order ID':
        f_order = dat['Order ID'] == target
        target = dat.where(f_order).groupby(['Order ID','Service Level','Carrier']).sum().reset_index()
        target['Size'] = data_length
    
    # 5 If the Data is based on Carrier.
    elif dat.columns[1] == 'Carrier':
        f_carrier = dat['level_0'] == target
        target = dat.where(f_carrier).groupby(['level_0','Carrier']).sum().reset_index()
        target['Size'] = data_length

    # print(type_sel,'\tSelected Target is ',target,' with the Probability of ',probability,'% [ Roulette Value : ',roulette_value," ]")
    return target

## 6.3 Objective/Fitness Function

In [9]:
def objective_function(res):

    lowest_cost = res[0][0]['Cost'].sum()
    lowest_cost_index = 0
    fitness = []

    for index_ocr,ocr in enumerate(res):
        if ocr[0]['Cost'].sum() < lowest_cost:
            lowest_cost = ocr[0]['Cost'].sum()
            lowest_cost_index = index_ocr

    for index_ocr,ocr in enumerate(res):
        fitness_value = 1/(ocr[0]['Cost'].sum()/lowest_cost)
        fitness.append(fitness_value)

    print('[ ',lowest_cost_index,' ] - LOWEST COST: $',lowest_cost)
    print('\nFitness')
    print(fitness)
    
    return lowest_cost,lowest_cost_index, fitness

## 6.4 Transportation Cost Function

In [11]:
def transportation_cost(selected_path_order,selected_carrier,dataOL_sub,dataFR_sub):
    
    transportation_cost = 0
    data_OL = dataOL_sub.loc[dataOL['Order ID'] == selected_path_order['Order ID'][0]].reset_index()
    data_FR = dataFR_sub.loc[selected_carrier['level_0'][0]]

    if data_OL['Weight'][0] < data_FR['minm_wgh_qty']:
        transportation_cost = 'Not Enough Weight'
    elif data_OL['Weight'][0] > data_FR['max_wgh_qty']:
        transportation_cost = 'Exceeded Weight'
        
    else:
        if data_FR['mode_dsc'] == 'GROUND':
            transportation_cost = (data_FR['rate']/data_FR['max_wgh_qty'])*data_OL['Weight'][0]
        else:
            transportation_cost = data_FR['rate']*data_OL['Weight'][0]

        if transportation_cost < data_FR['minimum cost']:
            transportation_cost = data_FR['minimum cost']

    return transportation_cost

## 6.5 Warehouse Cost and Capacity Function

In [12]:
def warehouse_cost(dataWhCost_sub,dataWhCap_sub,selected_path_order,selected_plant_code,dataOL_sub):

    data_Cap = dataWhCap_sub.loc[dataWhCap_sub['Plant ID'] == selected_plant_code].reset_index()
    data_Cap_index = dataWhCap_sub.loc[dataWhCap_sub['Plant ID'] == selected_plant_code]
    data_Cap_index = data_Cap_index.index[0]
    data_Cost = dataWhCost_sub.loc[dataWhCost_sub['WH'] == selected_plant_code].reset_index()
    data_OL = dataOL_sub.loc[dataOL_sub['Order ID'] == selected_path_order['Order ID'][0]].reset_index()
    wh_cost = 0

    if data_Cap['Daily Capacity '][0] > 0:
        wh_cost = data_OL['Unit quantity'][0]*data_Cost['Cost/unit'][0]

    else:
        wh_cost = 'Warehouse is Full!'

    return wh_cost

## 6.6 Pheromone Update Function 

In [13]:
def pheromone_update(res,fitness):

    for ft in range(len(fitness)):
        carrier_l = res[ft][0]['Carrier Index'].where(res[ft][0]['Carrier Index'] != 'No Carrier').dropna().tolist()
        order_l = res[ft][0]['Order ID'].tolist()

        for ol in range(len(order_l)):
            f_data_OL_index = dataOL[dataOL['Order ID']==order_l[ol]].index.values
            dataOL.at[f_data_OL_index,'Pheromone'] = dataOL['Pheromone'][f_data_OL_index] + fitness[2][ft]

        if carrier_l:
            for cl in range(len(carrier_l)):
                dataFR.at[carrier_l[cl],'Pheromone'] = dataFR['Pheromone'][int(carrier_l[cl])] + fitness[2][ft]

    return dataOL,dataFR

In [76]:
print(res[1][0])
meow = ['hiok']
if meow:
    print('meow')

     Port Plant Code      Order ID Service ID     Carrier Carrier Index  \
0  PORT09    PLANT16  1.447143e+09        CRF  No Carrier    No Carrier   
1  PORT09    PLANT16  1.447233e+09        CRF  No Carrier    No Carrier   
2  PORT05    PLANT04  1.447391e+09        CRF  No Carrier    No Carrier   

   Status  Pheromone         Cost  
0    True          0  1796.939827  
1    True          0  9238.113728  
2    True          0  9387.232874  
meow


## 6.7 Fresh DataFrames

In [14]:
def fresh_dataframes(OL, FR, WCap, WCost):

    dataOL, dataFR, dataWhCap, dataWhCost = OL, FR, WCap, WCost
    return dataOL, dataFR, dataWhCap, dataWhCost